This notebook uses the kdot accidents layer and AADT from the KDOT Khub publication database to calaulate crash rates using the equation

$$R = \frac{C x 100,000,000}{V x 365 x N x L}$$

The variables in this equation are:

R = crash rate for the road segment expressed as crashes per 100 million vehicle-miles of travel,

C = Total number of crashes in the study period

V = Traffic volumes using Average Annual Daily Traffic (AADT) volumes

N = Number of years of data

L = Length of the roadway segment in miles

https://safety.fhwa.dot.gov/local_rural/training/fhwasa1109/app_c.cfm

https://wfs.ksdot.org/arcgis_web_adaptor/rest/services/Transportation/Accidents/FeatureServer



In [6]:
projdb =r"C:\Users\kgonterwitz\OneDrive - City of Lawrence KS\Documents\ArcGIS\Projects\KDOT_CrashMaps\KDOT_CrashMaps.gdb"

In [10]:
#copy local Accidents data from KDOT Wfs for Douglas County to project database

with arcpy.EnvManager(scratchWorkspace=projdb, outputCoordinateSystem="PROJCS['NAD_1983_2011_KS_RCS_Zone_11',GEOGCS['GCS_NAD_1983_2011',DATUM['D_NAD_1983_2011',SPHEROID['GRS_1980',6378137.0,298.257222101]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Lambert_Conformal_Conic'],PARAMETER['False_Easting',11500000.0],PARAMETER['False_Northing',600000.0],PARAMETER['Central_Meridian',-95.25],PARAMETER['Standard_Parallel_1',39.1],PARAMETER['Scale_Factor',1.000033],PARAMETER['Latitude_Of_Origin',39.1],UNIT['Foot_US',0.3048006096012192]]", workspace=projdb):
    arcpy.conversion.FeatureClassToFeatureClass("Accidents", projdb, "Accidents_DG", "ACC_COUNTY = 'DOUGLAS'", '#', '')

In [8]:
#copy local AADT data from Khub publiation geodatabase to project database 
#selected are assigned routes to douglas county and routes longer than ten feet
#the minimum length threshold should be reviewed and possible raised
#if the minimum lenth threshod rises above 30 feet, raise spatial join parameter accordingly

arcpy.management.SelectLayerByAttribute("ev_AADT", "NEW_SELECTION", "RouteID LIKE '023%' And Shape_Length > 35", None)

with arcpy.EnvManager(outputCoordinateSystem="PROJCS['NAD_1983_2011_KS_RCS_Zone_11',GEOGCS['GCS_NAD_1983_2011',DATUM['D_NAD_1983_2011',SPHEROID['GRS_1980',6378137.0,298.257222101]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Lambert_Conformal_Conic'],PARAMETER['False_Easting',11500000.0],PARAMETER['False_Northing',600000.0],PARAMETER['Central_Meridian',-95.25],PARAMETER['Standard_Parallel_1',39.1],PARAMETER['Scale_Factor',1.000033],PARAMETER['Latitude_Of_Origin',39.1],UNIT['Foot_US',0.3048006096012192]]"):
    arcpy.conversion.FeatureClassToFeatureClass("ev_AADT", projdb, "AADT_2019", "RouteID LIKE '023%'", '#', '')

In [11]:
#this cell outputs the calculated crash rate for mapping and review with ten years of crash history plus the current year

arcpy.analysis.SpatialJoin("AADT_2019", "Accidents_DG", "SJ_Acc_AADT_2019", "JOIN_ONE_TO_ONE", "KEEP_ALL", '#', "INTERSECT", "36 Feet", '')
arcpy.management.CalculateField("SJ_Acc_AADT_2019", "CrashRateVolumetric", "!Join_Count! * 100000000/(!AADTCount! * 365 * 10 * (!ToMeasure!-!FromMeasure!))", "PYTHON3", '', "DOUBLE")

<Result 'SJ_Acc_AADT_2019'>

In [12]:
#pedestrian (and bicycle) related crashes included, 10 year average

arcpy.management.MakeFeatureLayer("Accidents_DG", "Accidents_BikePed", "ALL_PEDESTRIANS > 0 Or PEDESTRIAN_ACCS > 0 Or PEDAL_CYCLIST_ACCS > 0", None, "#")

arcpy.analysis.SpatialJoin("AADT_2019", "Accidents_BikePed", "SJ_AccBP_AADT_2019", "JOIN_ONE_TO_ONE", "KEEP_ALL", '#', "INTERSECT", "36 Feet", '')

arcpy.management.CalculateField("SJ_AccBP_AADT_2019", "PedCrashRateVolumetric", "!Join_Count! * 100000000/(!AADTCount! * 365 * 10 * (!ToMeasure!-!FromMeasure!))", "PYTHON3", '', "DOUBLE")

<Result 'SJ_AccBP_AADT_2019'>

compare rates to Kansas State Highway Safety Report 
https://www.fhwa.dot.gov/tpm/reporting/state/safety.cfm?state=Kansas
    
the Performance measures in the HSIP are total fataility rate, and total serious injury rate, and non-motorized fatalities and serious injuries

The same calculate as above will calculate the rates with modfication of the input crash data and the modification of the rate unit to per hundred million vmt.  The fatality rate will be the rate of fatality crashes not  the number of fatalities per fatal crash.  


In [19]:
#nonmotorized severe and fatal crash rates - 5 year history

arcpy.management.MakeFeatureLayer("Accidents_DG", "NonMotorized_Severe", "(ALL_PEDESTRIANS > 0 Or PEDESTRIAN_ACCS > 0 Or PEDAL_CYCLIST_ACCS > 0) AND ACC_YEAR IN ('2020', '2019', '2017', '2018', '2016') AND MOST_SERIOUS_INJURY IN ('D', 'F')", None, "#")

arcpy.analysis.SpatialJoin("AADT_2019", "NonMotorized_Severe", "SJ_AccNM_DF_AADT_2019", "JOIN_ONE_TO_ONE", "KEEP_ALL", '#', "INTERSECT", "36 Feet", '')

arcpy.management.CalculateField("SJ_AccNM_DF_AADT_2019", "NonMotorized_Severe_P100M", "!Join_Count! * 100000000/(!AADTCount! * 365 * 5 * (!ToMeasure!-!FromMeasure!))", "PYTHON3", '', "DOUBLE")

<Result 'SJ_AccNM_DF_AADT_2019'>

In [28]:
# fatality crash rate per 100 Million VMT - 5 year

arcpy.management.MakeFeatureLayer("Accidents_DG", "FatalCrash", "MOST_SERIOUS_INJURY IN ('F') AND ACC_YEAR IN ('2020', '2019', '2017', '2018', '2016')", None, "#")

arcpy.analysis.SpatialJoin("AADT_2019", "FatalCrash", "SJ_AccF_AADT_2019", "JOIN_ONE_TO_ONE", "KEEP_ALL", '#', "INTERSECT", "36 Feet", '')

arcpy.management.CalculateField("SJ_AccF_AADT_2019", "FatalityRate100VMT", "!Join_Count! * 100000000/(!AADTCount! * 365 * 5 * (!ToMeasure!-!FromMeasure!))", "PYTHON3", '', "DOUBLE")

<Result 'SJ_AccF_AADT_2019'>

In [29]:
# Serious Injury rate per 100 Million VMT (not including fatalities) - 5 year

arcpy.management.MakeFeatureLayer("Accidents_DG", "SeriousInjury", "MOST_SERIOUS_INJURY IN ('D') AND ACC_YEAR IN ('2020', '2019', '2017', '2018', '2016')", None, "#")

arcpy.analysis.SpatialJoin("AADT_2019", "SeriousInjury", "SJ_AccD_AADT_2019", "JOIN_ONE_TO_ONE", "KEEP_ALL", '#', "INTERSECT", "36 Feet", '')

arcpy.management.CalculateField("SJ_AccD_AADT_2019", "SeriousInjuryRate100VMT", "!Join_Count! * 100000000/(!AADTCount! * 365 * 5 * (!ToMeasure!-!FromMeasure!))", "PYTHON3", '', "DOUBLE")

<Result 'SJ_AccD_AADT_2019'>

KDOT Numbers
to gain context about average crash rates based on KDOT data, 
the following reports can be referenced:
    
https://www.ksdot.org/Assets/wwwksdotorg/bureaus/burTransPlan/prodinfo/Mileage_Travel/CountyMiles2017.pdf
    
1391 Miles of highway in douglas county in 2017    

https://www.ksdot.org/Assets/wwwksdotorg/bureaus/burTransPlan/prodinfo/Mileage_Travel/CountyDVMT2017.pdf

daily VMT in douglas county (2017)- 2,983,274

https://www.ksdot.org/bureaus/burTransPlan/prodinfo/Mileage_Travel/MileTravel2017.asp

Total Highway Miles in Kansas (2017) - 142,054 miles

Average Daily Travel in Kansas (2017) = 88,248,910 Vehicle Miles

from KDOT Crash data online - 

Douglas County:
Fatal Crashes - 43 between 2016-2020
Severe Injury Crashes - 108 between 2016-2020
Statewide there are about 380 fatal crashes per year
Statewide there are about 1000 serious injury crashes per year

In [30]:
print("DG Co Average AADT is " + str(2983274/1391))
print("Statewide Average AADT is " + str(88248910/142054))
print("DG Co Average 5 yr fatality rate is " + str(43*100000000/(2144*365*5*1391)))
print("DG Co Average 5 yr severe injury rate is " + str(108*100000000/(2144*365*5*1391)))
print("Statewide Average 5 yr fatality rate is " + str(380*5*100000000/(142054*365*5*621.25)))
print("Statewide Average 5 yr severe injury rate is " + str(1000*5*100000000/(142054*365*5*621.25)))

DG Co Average AADT is 2144.6973400431343
Statewide Average AADT is 621.2349529052332
DG Co Average 5 yr fatality rate is 0.790048359778763
DG Co Average 5 yr severe injury rate is 1.9843075082815442
Statewide Average 5 yr fatality rate is 1.1796980544746043
Statewide Average 5 yr severe injury rate is 3.1044685644068535
